<a href="https://colab.research.google.com/github/Anoif01/Traffic_Forecasting_STGCN/blob/main/All_Models_TrafficForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install -q wandb
!pip install mamba-ssm --no-build-isolation -q
from mamba_ssm import Mamba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout):
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:986: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:1045: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type=

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F

# from torchsummary import summary
# from Param import *

import os
import shutil
import json
import random
import sys
import math
import numpy as np
import pandas as pd
from scipy.sparse.linalg import eigs
import scipy.sparse as ss
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import time

# pandas progress bar and learning progress bar
from tqdm.notebook import tqdm
tqdm.pandas()
from IPython.display import display

import wandb
from argparse import Namespace
import gc

os.chdir('/content/drive/MyDrive/Personal_Project/Traffic_Prediction/')
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

# Read credentiels
from google.colab import userdata
WANDB_TOKEN = userdata.get('WANDB_KEY')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Functions/Classes Def


## 1. Dataset Func


In [4]:
# Function to set random seed for reproducibility
def set_seed(seed):
    """
    Sets the random seed for Python's built-in random module, NumPy, and PyTorch
    to ensure reproducibility across experiments.

    Parameters:
    seed (int): The seed value to be used for generating random numbers.
    """

    # Set the seed for Python's hash-based operations
    os.environ['PYTHONHASHSEED'] = str(seed)

    # Set the seed for the Python random module
    random.seed(seed)

    # Set the seed for NumPy's random number generator
    np.random.seed(seed)

    # Set the seed for PyTorch's random number generator
    torch.manual_seed(seed)

    # Ensure PyTorch uses deterministic algorithms (may affect performance)
    torch.use_deterministic_algorithms(True)

In [5]:
def getXSYS_single(data, mode):
    """
    Splits the dataset into input (XS) and output (YS) sequences for training or testing.

    Parameters:
    data (np.array): The complete dataset as a NumPy array.
    mode (str): Specifies whether to generate 'TRAIN' or 'TEST' sequences.

    Returns:
    XS (np.array): Input sequences of shape (num_samples, 1, TIMESTEP_IN, num_features).
    YS (np.array): Output sequences of shape (num_samples, 1, 1, num_features).
    """

    # Number of data points allocated for training
    TRAIN_NUM = int(data.shape[0] * TRAINRATIO)
    XS, YS = [], []

    if mode == 'TRAIN':
        # Generate training sequences
        # Iterate over the data to create input-output pairs
        for i in range(TRAIN_NUM - TIMESTEP_OUT - TIMESTEP_IN + 1):
            # Input sequence (historical data)
            x = data[i:i+TIMESTEP_IN, :]
            # Output sequence (data to be predicted)
            y = data[i+TIMESTEP_IN+TIMESTEP_OUT-1:i+TIMESTEP_IN+TIMESTEP_OUT, :]
            XS.append(x)
            YS.append(y)

    elif mode == 'TEST':
        # Generate testing sequences
        # Iterate over the data, starting after the training portion, to create input-output pairs
        for i in range(TRAIN_NUM - TIMESTEP_IN,  data.shape[0] - TIMESTEP_OUT - TIMESTEP_IN + 1):
            # Input sequence (historical data)
            x = data[i:i+TIMESTEP_IN, :]
            # Output sequence (data to be predicted)
            y = data[i+TIMESTEP_IN+TIMESTEP_OUT-1:i+TIMESTEP_IN+TIMESTEP_OUT, :]
            XS.append(x)
            YS.append(y)

    # Convert lists to NumPy arrays
    XS, YS = np.array(XS), np.array(YS)
    # Add an extra dimension for the convolutional channel
    XS, YS = XS[:, np.newaxis, :, :], YS[:, np.newaxis, :]

    return XS, YS

In [6]:
def create_dataset(XS, YS, device, mode):
    if mode == "TRAIN_VALID":
        # add XS array and YS array on GPU device
        XS_torch, YS_torch = torch.Tensor(XS).to(device), torch.Tensor(YS).to(device)

        # create train_valid Dataset
        trainval_data = torch.utils.data.TensorDataset(XS_torch, YS_torch)
        trainval_size = len(trainval_data)

        # calculate the real train size
        train_size = int(trainval_size * (1-TRAINVALSPLIT))

        # gather train data and valid data
        train_data = torch.utils.data.Subset(trainval_data, list(range(0, train_size)))
        val_data = torch.utils.data.Subset(trainval_data, list(range(train_size, trainval_size)))

        print(f"Original Train_Valid Dataset size: {trainval_size}.")
        print(f"Train Dataset size: {len(train_data)}.")
        print(f"Valide Dataset size: {len(val_data)}.")

        # create the dataloaders
        train_iter = torch.utils.data.DataLoader(train_data, BATCHSIZE, shuffle=True)
        val_iter = torch.utils.data.DataLoader(val_data, BATCHSIZE, shuffle=True)
        return train_iter, val_iter

    elif mode == "TEST":
        # add XS array and YS array on GPU device
        XS_torch, YS_torch = torch.Tensor(XS).to(device), torch.Tensor(YS).to(device)

        # create train_valid Dataset
        test_data = torch.utils.data.TensorDataset(XS_torch, YS_torch)
        test_size = len(test_data)
        print(f"Test Dataset size: {test_size}.")

        # create the dataloaders
        test_iter = torch.utils.data.DataLoader(test_data, BATCHSIZE, shuffle=False)
        return test_iter

## 2 Evaluate

In [7]:
def evaluate_func(y_true, y_pred, precision=10):
    # print('MSE:', round(MSE(y_true, y_pred), precision))
    # print('RMSE:', round(RMSE(y_true, y_pred), precision))
    # print('MAE:', round(MAE(y_true, y_pred), precision))
    # print('MAPE:', round(MAPE(y_true, y_pred), precision), '%')
    # print('PCC:', round(PCC(y_true, y_pred), precision))
    return MSE_func(y_true, y_pred), RMSE_func(y_true, y_pred), MAE_func(y_true, y_pred), MAPE_func(y_true, y_pred)

def MSE_func(y_true, y_pred):
    y_true[y_true < 1e-5] = 0
    y_pred[y_pred < 1e-5] = 0
    with np.errstate(divide = 'ignore', invalid = 'ignore'):
        mask = np.not_equal(y_true, 0)
        mask = mask.astype(np.float32)
        mask /= np.mean(mask)
        mse = np.square(y_pred - y_true)
        mse = np.nan_to_num(mse * mask)
        mse = np.mean(mse)
        return mse

def RMSE_func(y_true, y_pred):
    y_true[y_true < 1e-5] = 0
    y_pred[y_pred < 1e-5] = 0
    with np.errstate(divide = 'ignore', invalid = 'ignore'):
        mask = np.not_equal(y_true, 0)
        mask = mask.astype(np.float32)
        mask /= np.mean(mask)
        rmse = np.square(np.abs(y_pred - y_true))
        rmse = np.nan_to_num(rmse * mask)
        rmse = np.sqrt(np.mean(rmse))
        return rmse

def MAE_func(y_true, y_pred):
    y_true[y_true < 1e-5] = 0
    y_pred[y_pred < 1e-5] = 0
    with np.errstate(divide = 'ignore', invalid = 'ignore'):
        mask = np.not_equal(y_true, 0)
        mask = mask.astype(np.float32)
        mask /= np.mean(mask)
        mae = np.abs(y_pred - y_true)
        mae = np.nan_to_num(mae * mask)
        mae = np.mean(mae)
        return mae

def MAPE_func(y_true, y_pred, null_val=0):
    y_true[y_true < 1e-5] = 0
    y_pred[y_pred < 1e-5] = 0
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(y_true)
        else:
            mask = np.not_equal(y_true, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        mape = np.abs(np.divide((y_pred - y_true).astype('float32'), y_true))
        mape = np.nan_to_num(mask * mape)
        return np.mean(mape) * 100

In [8]:
def evaluateModel(model, criterion, data_iter):
    model.eval()
    l_sum, n = 0.0, 0
    with torch.no_grad():
        for x, y in data_iter:
            y_pred = model(x)
            l = criterion(y_pred, y)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        return l_sum / n

def predictModel(model, data_iter):
    YS_pred = []
    model.eval()
    with torch.no_grad():
        for x, y in data_iter:
            YS_pred_batch = model(x)
            YS_pred_batch = YS_pred_batch.cpu().numpy()
            YS_pred.append(YS_pred_batch)
        YS_pred = np.vstack(YS_pred)
    return YS_pred

## 3. Adjacent Mat

In [9]:
def weight_matrix(W, sigma2=0.1, epsilon=0.5):
    """
    Computes a weighted adjacency matrix with controlled sparsity.

    Parameters:
    W (np.ndarray): The original adjacency matrix of shape [n_route, n_route].
    sigma2 (float): The variance scalar applied to the matrix W (default is 0.1).
    epsilon (float): Threshold to control the sparsity of the matrix W (default is 0.5).

    Returns:
    np.ndarray: The transformed weight matrix of shape [n_route, n_route],
                with sparsity controlled by epsilon and zeros on the diagonal.
    """

    n = W.shape[0]  # Number of nodes/routes

    # Normalize the adjacency matrix by scaling down its values
    W = W / 10000

    # Replace zeros (representing no connection) with infinity
    W[W == 0] = np.inf

    # Square the matrix values
    W2 = W * W

    # Create a mask to remove self-loops (diagonal elements)
    W_mask = (np.ones([n, n]) - np.identity(n))

    # Apply the Gaussian kernel and sparsity threshold, then apply the mask
    return np.exp(-W2 / sigma2) * (np.exp(-W2 / sigma2) >= epsilon) * W_mask

def scaled_laplacian(A):
    """
    Computes the scaled Laplacian matrix of an adjacency matrix A.

    Parameters:
    A (np.ndarray): The adjacency matrix of shape [n, n].

    Returns:
    np.ndarray: The scaled Laplacian matrix of shape [n, n].
    """

    n = A.shape[0]  # Number of nodes

    # Degree matrix: diagonal matrix where each diagonal element is the sum of the corresponding row in A
    d = np.sum(A, axis=1)

    # Unnormalized Laplacian matrix: L = D - A
    L = np.diag(d) - A

    # Normalize the Laplacian matrix
    for i in range(n):
        for j in range(n):
            if d[i] > 0 and d[j] > 0:
                # Normalize by the degree of nodes i and j
                L[i, j] /= np.sqrt(d[i] * d[j])

    # Compute the largest eigenvalue of the Laplacian matrix
    lam = np.linalg.eigvals(L).max().real

    # Scale the Laplacian matrix by the largest eigenvalue and subtract the identity matrix
    return 2 * L / lam - np.eye(n)

def cheb_poly(L, Ks):
    """
    Computes the Chebyshev polynomials up to the Ks-th order for a given Laplacian matrix L.

    Parameters:
    L (np.ndarray): The scaled Laplacian matrix of shape [n, n].
    Ks (int): The order up to which Chebyshev polynomials are computed.

    Returns:
    np.ndarray: A tensor of Chebyshev polynomials with shape [Ks, n, n].
    """

    n = L.shape[0]  # Number of nodes

    # Initialize the list of Chebyshev polynomials
    LL = [np.eye(n), L[:]]  # T_0(L) = I, T_1(L) = L

    # Compute higher-order Chebyshev polynomials iteratively
    for i in range(2, Ks):
        # T_k(L) = 2 * L * T_{k-1}(L) - T_{k-2}(L)
        LL.append(np.matmul(2 * L, LL[-1]) - LL[-2])

    # Convert the list of polynomials to a NumPy array
    return np.asarray(LL)

## 4. Model Design

### 4.1 temporal_conv_layer

In [10]:
class align(nn.Module):
    """
    The align module is designed to reshape the input tensor's channel dimensions.
    Depending on whether the input channels are greater or less than the output channels,
    this module either reduces or increases the number of channels.

    Parameters:
    c_in (int): Number of input channels.
    c_out (int): Number of output channels.
    """

    def __init__(self, c_in, c_out):
        super(align, self).__init__()
        self.c_in = c_in
        self.c_out = c_out

        # If the number of input channels (c_in) is greater than the output channels (c_out),
        # reduce the channel size using a 1x1 convolution. This acts as a linear transformation
        # across the channel dimension.
        if c_in > c_out:
            self.conv1x1 = nn.Conv2d(c_in, c_out, 1)

    def forward(self, x):
        # If the input channels are greater than the output channels, apply the 1x1 convolution
        # to reduce the number of channels.
        if self.c_in > self.c_out:
            return self.conv1x1(x)

        # If the input channels are less than the output channels, pad the input tensor
        # to increase the number of channels. The padding is added to the second dimension
        # (channels) while keeping other dimensions unchanged.
        if self.c_in < self.c_out:
            # Pad format: [pad_left, pad_right, pad_top, pad_bottom, pad_front, pad_back, ...]
            return F.pad(x, [0, 0, 0, 0, 0, self.c_out - self.c_in, 0, 0])

        # If the input and output channels are the same, return the input tensor unchanged.
        return x

In [11]:
class temporal_conv_layer(nn.Module):
    """
    Temporal Convolution Layer designed to extract spatio-temporal features.
    This layer can use either Gated Linear Units (GLU) or traditional activation functions
    like sigmoid or ReLU.

    Parameters:
    kt (int): The size of the temporal convolution kernel.
    c_in (int): The number of input channels.
    c_out (int): The number of output channels.
    act (str): The activation function to be used ("GLU", "sigmoid", or "relu").
    """

    def __init__(self, kt, c_in, c_out, act="relu"):
        super(temporal_conv_layer, self).__init__()
        self.kt = kt  # Temporal kernel size
        self.act = act  # Activation function
        self.c_out = c_out  # Number of output channels

        # Align input channels to output channels if necessary
        self.align = align(c_in, c_out)

        # Define the convolution layer
        # The kernel size is (kt, 1), which implies a 1D convolution over the temporal dimension
        if self.act == "GLU":
            # GLU requires double the number of output channels
            self.conv = nn.Conv2d(c_in, c_out * 2, (kt, 1), 1)
        else:
            self.conv = nn.Conv2d(c_in, c_out, (kt, 1), 1)

    def forward(self, x):
        # Align the input to have the same number of channels as the output
        # Only take the last `kt-1` time steps from the aligned input
        x_in = self.align(x)[:, :, self.kt-1:, :]

        if self.act == "GLU":
            # Apply the convolution, then use GLU activation
            x_conv = self.conv(x)
            # GLU: (x_conv * sigmoid(x_conv)) after adding residual connection (x_in)
            return (x_conv[:, :self.c_out, :, :] + x_in) * torch.sigmoid(x_conv[:, self.c_out:, :, :])

        if self.act == "sigmoid":
            # Apply the convolution and sigmoid activation
            return torch.sigmoid(self.conv(x) + x_in)

        # Default ReLU activation for positive linear unit activation
        return torch.relu(self.conv(x) + x_in)

In [13]:
class dilated_temporal_conv_layer(nn.Module):
    """
    Dilated Temporal Convolution Layer designed to extract spatio-temporal features
    using dilated convolutions. This layer can use either Gated Linear Units (GLU) or
    traditional activation functions like sigmoid or ReLU.

    Parameters:
    kt (int): The size of the temporal convolution kernel.
    c_in (int): The number of input channels.
    c_out (int): The number of output channels.
    act (str): The activation function to be used ("GLU", "sigmoid", or "relu").
    """

    def __init__(self, kt, c_in, c_out, act="relu"):
        super(dilated_temporal_conv_layer, self).__init__()
        self.kt = kt  # Temporal kernel size
        self.act = act  # Activation function
        self.c_out = c_out  # Number of output channels

        # Align input channels to output channels if necessary
        self.align = align(c_in, c_out)

        # Define the dilated convolution layer
        # The kernel size is (kt, 1), implying a 1D convolution over the temporal dimension
        # Padding and dilation are used to expand the receptive field
        if self.act == "GLU":
            # For GLU, double the number of output channels
            # Padding is set to (2, 0) to ensure the output size matches the input size
            # Dilation expands the kernel's receptive field
            self.conv = nn.Conv2d(c_in, c_out * 2, (kt, 1), 1, padding=(2, 0), dilation=2)
        else:
            self.conv = nn.Conv2d(c_in, c_out, (kt, 1), 1, padding=(2, 0), dilation=2)

    def forward(self, x):
        # Align the input to have the same number of channels as the output
        x_in = self.align(x)  # Align channels

        if self.act == "GLU":
            # Apply the convolution, then use GLU activation
            x_conv = self.conv(x)
            # GLU: (x_conv * sigmoid(x_conv)) after adding residual connection (x_in)
            return (x_conv[:, :self.c_out, :, :] + x_in) * torch.sigmoid(x_conv[:, self.c_out:, :, :])

        if self.act == "sigmoid":
            # Apply the convolution and sigmoid activation
            return torch.sigmoid(self.conv(x) + x_in)

        # Default ReLU activation for positive linear unit activation
        return torch.relu(self.conv(x) + x_in)

In [12]:
class dilated_no0_temporal_conv_layer(nn.Module):
    """
    Dilated Temporal Convolution Layer without zero-padding, designed to extract
    spatio-temporal features using dilated convolutions. This layer can use either
    Gated Linear Units (GLU) or traditional activation functions like sigmoid or ReLU.

    Parameters:
    kt (int): The size of the temporal convolution kernel.
    c_in (int): The number of input channels.
    c_out (int): The number of output channels.
    act (str): The activation function to be used ("GLU", "sigmoid", or "relu").
    """

    def __init__(self, kt, c_in, c_out, act="relu"):
        super(dilated_no0_temporal_conv_layer, self).__init__()
        self.kt = kt  # Temporal kernel size
        self.act = act  # Activation function
        self.c_out = c_out  # Number of output channels

        # Align input channels to output channels if necessary
        self.align = align(c_in, c_out)

        # Define the dilated convolution layer
        # The kernel size is (kt, 1), implying a 1D convolution over the temporal dimension
        # Dilation expands the receptive field
        if self.act == "GLU":
            # For GLU, double the number of output channels
            self.conv = nn.Conv2d(c_in, c_out * 2, (kt, 1), 1, dilation=2)
        else:
            self.conv = nn.Conv2d(c_in, c_out, (kt, 1), 1, dilation=2)

    def forward(self, x):
        # Align the input to have the same number of channels as the output
        x_in = self.align(x)  # Align channels

        # Apply padding using the 'replicate' mode to pad the third dimension.
        # Here, padding is applied to the third dimension, ensuring no zeros are introduced.
        pad_x = F.pad(x, [0, 0, 0, self.kt + 2 - 1], mode="replicate")

        if self.act == "GLU":
            # Apply the convolution, then use GLU activation
            x_conv = self.conv(pad_x)
            # GLU: (x_conv * sigmoid(x_conv)) after adding residual connection (x_in)
            return (x_conv[:, :self.c_out, :, :] + x_in) * torch.sigmoid(x_conv[:, self.c_out:, :, :])

        if self.act == "sigmoid":
            # Apply the convolution and sigmoid activation
            return torch.sigmoid(self.conv(pad_x) + x_in)

        # Default ReLU activation for positive linear unit activation
        return torch.relu(self.conv(pad_x) + x_in)

In [14]:
class temporal_lstm_layer(nn.Module):
    """
    Temporal LSTM Layer designed to extract spatio-temporal features using LSTM units.
    This layer supports Gated Linear Units (GLU), sigmoid, or ReLU as activation functions.

    Parameters:
    c_in (int): The number of input channels.
    c_out (int): The number of output channels.
    T_IN (int): The number of input time steps.
    BATCHSIZE (int): The batch size.
    N_NODE (int): The number of nodes (or spatial dimension).
    act (str): The activation function to be used ("GLU", "sigmoid", or "relu").
    """

    def __init__(self, c_in, c_out, T_IN, BATCHSIZE, N_NODE, act="relu"):
        super(temporal_lstm_layer, self).__init__()
        self.act = act  # Activation function
        self.c_in = c_in  # Number of input channels
        self.c_out = c_out  # Number of output channels
        self.T_IN = T_IN  # Number of input time steps
        self.BATCHSIZE = BATCHSIZE  # Batch size
        self.N_NODE = N_NODE  # Number of nodes (spatial dimension)

        # Align input channels to output channels if necessary
        self.align = align(c_in, c_out)

        # Select LSTM configuration based on the type of LSTM (standard or bidirectional) and activation function
        if is_STG_LSTM:
            if self.act == "GLU":
                # LSTM with GLU activation, doubling the output channels
                self.lstm = nn.LSTM(input_size=c_in, hidden_size=c_out * 2, num_layers=1, batch_first=False)
            else:
                # Standard LSTM
                self.lstm = nn.LSTM(input_size=c_in, hidden_size=c_out, num_layers=1, batch_first=False)

        elif is_STG_BiLSTM:
            if self.act == "GLU":
                # Bidirectional LSTM with GLU activation
                self.lstm = nn.LSTM(input_size=c_in, hidden_size=c_out, num_layers=1, bidirectional=True, batch_first=False)
            else:
                # Bidirectional LSTM with reduced output channels
                self.lstm = nn.LSTM(input_size=c_in, hidden_size=int(c_out / 2), num_layers=1, bidirectional=True, batch_first=False)

    def forward(self, x):
        # Align the input to have the same number of channels as the output
        x_in = self.align(x)  # Aligned input shape: [batch_size, c_out, T_IN, N_NODE]

        # Permute dimensions to prepare for LSTM (time step dimension first)
        x_permut = x.permute(2, 0, 3, 1)  # Shape: [T_IN, batch_size, N_NODE, c_out]
        x_reshape = torch.reshape(x_permut, (self.T_IN, -1, self.c_in))  # Shape: [T_IN, batch_size*N_NODE, c_out]

        if self.act == "GLU":
            # LSTM with GLU activation
            x_lstm, _ = self.lstm(x_reshape)
            x_lstm = x_lstm.reshape(self.T_IN, -1, self.N_NODE, self.c_out * 2).permute(1, 3, 0, 2)
            x_out = (x_lstm[:, :self.c_out, :, :] + x_in) * torch.sigmoid(x_lstm[:, self.c_out:, :, :])
            return x_out

        if self.act == "sigmoid":
            # LSTM with sigmoid activation
            x_lstm, _ = self.lstm(x_reshape)
            x_lstm = x_lstm.reshape(self.T_IN, -1, self.N_NODE, self.c_out).permute(1, 3, 0, 2)
            x_out = torch.sigmoid(x_lstm) + x_in
            return x_out

        # Default ReLU activation for LSTM
        x_lstm, _ = self.lstm(x_reshape)
        x_lstm = x_lstm.reshape(self.T_IN, -1, self.N_NODE, self.c_out).permute(1, 3, 0, 2)
        x_out = torch.relu(x_lstm) + x_in
        return x_out

In [15]:
class attention_lstm_layer(nn.Module):
    """
    Attention LSTM Layer designed to extract spatio-temporal features using LSTM units combined with attention mechanisms.
    This layer supports Gated Linear Units (GLU), sigmoid, or ReLU as activation functions.

    Parameters:
    c_in (int): The number of input channels.
    c_out (int): The number of output channels.
    T_IN (int): The number of input time steps.
    BATCHSIZE (int): The batch size.
    N_NODE (int): The number of nodes (or spatial dimension).
    act (str): The activation function to be used ("GLU", "sigmoid", or "relu").
    """

    def __init__(self, c_in, c_out, T_IN, BATCHSIZE, N_NODE, act="relu"):
        super(attention_lstm_layer, self).__init__()
        self.act = act  # Activation function
        self.c_in = c_in  # Number of input channels
        self.c_out = c_out  # Number of output channels
        self.T_IN = T_IN  # Number of input time steps
        self.BATCHSIZE = BATCHSIZE  # Batch size
        self.N_NODE = N_NODE  # Number of nodes (spatial dimension)

        # Configure LSTM and attention layers based on the type of LSTM (standard or bidirectional) and activation function
        if is_STG_AttentionLSTM:
            self.align = align(c_in, c_out)  # Align input channels to output channels
            if self.act == "GLU":
                self.lstm = nn.LSTM(input_size=c_in, hidden_size=c_out * 2, num_layers=1, batch_first=False)
                self.attention = nn.Linear(c_out, 1)
            else:
                self.lstm = nn.LSTM(input_size=c_in, hidden_size=c_out, num_layers=1, batch_first=False)
                self.attention = nn.Linear(c_out, 1)
        elif is_STG_BiAttentionLSTM:
            self.align = align(c_in, c_out)  # Align input channels to output channels
            if self.act == "GLU":
                self.lstm = nn.LSTM(input_size=c_in, hidden_size=c_out, num_layers=1, bidirectional=True, batch_first=False)
                self.attention = nn.Linear(c_out, 1)
            else:
                self.lstm = nn.LSTM(input_size=c_in, hidden_size=int(c_out / 2), num_layers=1, bidirectional=True, batch_first=False)
                self.attention = nn.Linear(c_out, 1)

    def forward(self, x):
        # Align the input to have the same number of channels as the output
        x_in = self.align(x)  # Shape: [batch_size, c_out, T_IN, N_NODE]

        # Permute dimensions to prepare for LSTM (time step dimension first)
        x_permut = x.permute(2, 0, 3, 1)  # Shape: [T_IN, batch_size, N_NODE, c_out]
        x_reshape = torch.reshape(x_permut, (self.T_IN, -1, self.c_in))  # Shape: [T_IN, batch_size*N_NODE, c_in]

        if self.act == "GLU":
            # LSTM with GLU activation
            x_lstm, _ = self.lstm(x_reshape)  # Shape: [T_IN, batch_size*N_NODE, 2*c_out]
            x_lstm = x_lstm.reshape(self.T_IN, -1, self.N_NODE, self.c_out * 2).permute(1, 3, 0, 2)
            x_out = (x_lstm[:, :self.c_out, :, :] + x_in) * torch.sigmoid(x_lstm[:, self.c_out:, :, :])  # Shape: [batch_size, c_out, T_IN, N_NODE]

            # Compute attention weights along the time dimension
            x_out = x_out.permute(0, 2, 3, 1)  # Shape: [batch_size, T_IN, N_NODE, c_out]
            attn_weights = torch.softmax(self.attention(x_out), dim=1)  # Shape: [batch_size, T_IN, N_NODE, 1]
            weighted_output = x_out * attn_weights  # Shape: [batch_size, T_IN, N_NODE, c_out]
            attn_output = torch.sum(weighted_output, dim=1)  # Shape: [batch_size, N_NODE, c_out]

            x_out = x_out.permute(0, 3, 1, 2)  # Shape: [batch_size, c_out, T_IN, N_NODE]
            return x_out, attn_output

        if self.act == "sigmoid":
            # LSTM with sigmoid activation
            x_lstm, _ = self.lstm(x_reshape)
            x_lstm = x_lstm.reshape(self.T_IN, -1, self.N_NODE, self.c_out).permute(1, 3, 0, 2)
            x_out = torch.sigmoid(x_lstm) + x_in
            x_out = x_out.permute(0, 2, 3, 1)

            # Compute attention weights and apply them
            attn_weights = torch.softmax(self.attention(x_out), dim=1)
            weighted_output = x_out * attn_weights
            attn_output = torch.sum(weighted_output, dim=1)

            x_out = x_out.permute(0, 3, 1, 2)  # Shape: [batch_size, c_out, T_IN, N_NODE]
            return x_out, attn_output

        # Default ReLU activation for LSTM
        x_lstm, _ = self.lstm(x_reshape)
        x_lstm = x_lstm.reshape(self.T_IN, -1, self.N_NODE, self.c_out).permute(1, 3, 0, 2)
        x_out = torch.relu(x_lstm) + x_in
        x_out = x_out.permute(0, 2, 3, 1)

        # Compute attention weights and apply them
        attn_weights = torch.softmax(self.attention(x_out), dim=1)
        weighted_output = x_out * attn_weights
        attn_output = torch.sum(weighted_output, dim=1)

        x_out = x_out.permute(0, 3, 1, 2)  # Shape: [batch_size, c_out, T_IN, N_NODE]
        return x_out, attn_output

In [16]:
class temporal_gru_layer(nn.Module):
    """
    Temporal GRU Layer designed to extract spatio-temporal features using GRU (Gated Recurrent Unit) units.
    This layer supports Gated Linear Units (GLU), sigmoid, or ReLU as activation functions.

    Parameters:
    c_in (int): The number of input channels.
    c_out (int): The number of output channels.
    T_IN (int): The number of input time steps.
    BATCHSIZE (int): The batch size.
    N_NODE (int): The number of nodes (or spatial dimension).
    act (str): The activation function to be used ("GLU", "sigmoid", or "relu").
    """

    def __init__(self, c_in, c_out, T_IN, BATCHSIZE, N_NODE, act="relu"):
        super(temporal_gru_layer, self).__init__()
        self.act = act  # Activation function
        self.c_in = c_in  # Number of input channels
        self.c_out = c_out  # Number of output channels
        self.T_IN = T_IN  # Number of input time steps
        self.BATCHSIZE = BATCHSIZE  # Batch size
        self.N_NODE = N_NODE  # Number of nodes (spatial dimension)
        self.align = align(c_in, c_out)  # Align input channels to output channels

        # Configure GRU based on the type of GRU (standard or bidirectional) and activation function
        if is_STG_GRU:
            if self.act == "GLU":
                self.gru = nn.GRU(input_size=c_in, hidden_size=c_out * 2, num_layers=1, batch_first=False)
            else:
                self.gru = nn.GRU(input_size=c_in, hidden_size=c_out, num_layers=1, batch_first=False)
        elif is_STG_BiGRU:
            if self.act == "GLU":
                self.gru = nn.GRU(input_size=c_in, hidden_size=c_out, num_layers=1, bidirectional=True, batch_first=False)
            else:
                self.gru = nn.GRU(input_size=c_in, hidden_size=int(c_out / 2), num_layers=1, bidirectional=True, batch_first=False)

    def forward(self, x):
        # Align the input to have the same number of channels as the output
        x_in = self.align(x)  # Shape: [batch_size, c_out, T_IN, N_NODE]

        # Permute dimensions to prepare for GRU (time step dimension first)
        x_permut = x.permute(2, 0, 3, 1)  # Shape: [T_IN, batch_size, N_NODE, c_out]
        x_reshape = torch.reshape(x_permut, (self.T_IN, -1, self.c_in))  # Shape: [T_IN, batch_size*N_NODE, c_in]

        if self.act == "GLU":
            # GRU with GLU activation
            x_gru, _ = self.gru(x_reshape)
            x_gru = x_gru.reshape(self.T_IN, -1, self.N_NODE, self.c_out * 2).permute(1, 3, 0, 2)
            x_out = (x_gru[:, :self.c_out, :, :] + x_in) * torch.sigmoid(x_gru[:, self.c_out:, :, :])
            return x_out

        if self.act == "sigmoid":
            # GRU with sigmoid activation
            x_gru, _ = self.gru(x_reshape)
            x_gru = x_gru.reshape(self.T_IN, -1, self.N_NODE, self.c_out).permute(1, 3, 0, 2)
            x_out = torch.sigmoid(x_gru) + x_in
            return x_out

        # Default ReLU activation for GRU
        x_gru, _ = self.gru(x_reshape)
        x_gru = x_gru.reshape(self.T_IN, -1, self.N_NODE, self.c_out).permute(1, 3, 0, 2)
        x_out = torch.relu(x_gru) + x_in
        return x_out


In [ ]:
class attention_gru_layer(nn.Module):
    """
    Attention-based GRU Layer designed to extract spatio-temporal features with attention mechanism.
    This layer supports Gated Linear Units (GLU), sigmoid, or ReLU as activation functions.

    Parameters:
    c_in (int): The number of input channels.
    c_out (int): The number of output channels.
    T_IN (int): The number of input time steps.
    BATCHSIZE (int): The batch size.
    N_NODE (int): The number of nodes (or spatial dimension).
    act (str): The activation function to be used ("GLU", "sigmoid", or "relu").
    """

    def __init__(self, c_in, c_out, T_IN, BATCHSIZE, N_NODE, act="relu"):
        super(attention_gru_layer, self).__init__()
        self.act = act  # Activation function
        self.c_in = c_in  # Number of input channels
        self.c_out = c_out  # Number of output channels
        self.T_IN = T_IN  # Number of input time steps
        self.BATCHSIZE = BATCHSIZE  # Batch size
        self.N_NODE = N_NODE  # Number of nodes (spatial dimension)

        # Configure GRU based on the type of GRU (standard or bidirectional) and activation function
        if is_STG_AttentionGRU:
            self.align = align(c_in, c_out)
            if self.act == "GLU":
                self.gru = nn.GRU(input_size=c_in, hidden_size=c_out * 2, num_layers=1, batch_first=False)
            else:
                self.gru = nn.GRU(input_size=c_in, hidden_size=c_out, num_layers=1, batch_first=False)
            self.attention = nn.Linear(c_out, 1)

        elif is_STG_BiAttentionGRU:
            self.align = align(c_in, c_out)
            if self.act == "GLU":
                self.gru = nn.GRU(input_size=c_in, hidden_size=c_out, num_layers=1, bidirectional=True, batch_first=False)
            else:
                self.gru = nn.GRU(input_size=c_in, hidden_size=int(c_out / 2), num_layers=1, bidirectional=True, batch_first=False)
            self.attention = nn.Linear(c_out, 1)

    def forward(self, x):
        # Align the input to have the same number of channels as the output
        x_in = self.align(x)  # Shape: [batch_size, c_out, T_IN, N_NODE]

        # Permute dimensions to prepare for GRU (time step dimension first)
        x_permut = x.permute(2, 0, 3, 1)  # Shape: [T_IN, batch_size, N_NODE, c_out]
        x_reshape = torch.reshape(x_permut, (self.T_IN, -1, self.c_in))  # Shape: [T_IN, batch_size*N_NODE, c_in]

        if self.act == "GLU":
            # GRU with GLU activation
            x_gru, _ = self.gru(x_reshape)  # Shape: [T_IN, batch_size*N_NODE, 2*c_out]
            x_gru = x_gru.reshape(self.T_IN, -1, self.N_NODE, self.c_out * 2).permute(1, 3, 0, 2)
            x_out = (x_gru[:, :self.c_out, :, :] + x_in) * torch.sigmoid(x_gru[:, self.c_out:, :, :])

            # Calculate attention weights across the time steps
            x_out = x_out.permute(0, 2, 3, 1)  # Shape: [batch_size, T_IN, N_NODE, c_out]
            attn_weights = torch.softmax(self.attention(x_out), dim=1)  # Shape: [batch_size, T_IN, N_NODE, 1]

            # Apply attention weights (weighted average)
            weighted_output = x_out * attn_weights  # Shape: [batch_size, T_IN, N_NODE, c_out]

            # Sum the weighted output to get the final attention representation
            attn_output = torch.sum(weighted_output, dim=1)  # Shape: [batch_size, N_NODE, c_out]

            # Permute back to original shape
            x_out = x_out.permute(0, 3, 1, 2)  # Shape: [batch_size, c_out, T_IN, N_NODE]
            return x_out, attn_output

        if self.act == "sigmoid":
            # GRU with sigmoid activation
            x_gru, _ = self.gru(x_reshape)
            x_gru = x_gru.reshape(self.T_IN, -1, self.N_NODE, self.c_out).permute(1, 3, 0, 2)
            x_out = torch.sigmoid(x_gru) + x_in
            x_out = x_out.permute(0, 2, 3, 1)

            # Calculate attention weights
            attn_weights = torch.softmax(self.attention(x_out), dim=1)

            # Apply attention weights (weighted average)
            weighted_output = x_out * attn_weights

            # Sum the weighted output to get the final attention representation
            attn_output = torch.sum(weighted_output, dim=1)

            # Permute back to original shape
            x_out = x_out.permute(0, 3, 1, 2)  # Shape: [batch_size, c_out, T_IN, N_NODE]
            return x_out, attn_output

        # Default ReLU activation for GRU
        x_gru, _ = self.gru(x_reshape)
        x_gru = x_gru.reshape(self.T_IN, -1, self.N_NODE, self.c_out).permute(1, 3, 0, 2)
        x_out = torch.relu(x_gru) + x_in
        x_out = x_out.permute(0, 2, 3, 1)

        # Calculate attention weights
        attn_weights = torch.softmax(self.attention(x_out), dim=1)

        # Apply attention weights (weighted average)
        weighted_output = x_out * attn_weights

        # Sum the weighted output to get the final attention representation
        attn_output = torch.sum(weighted_output, dim=1)

        # Permute back to original shape
        x_out = x_out.permute(0, 3, 1, 2)  # Shape: [batch_size, c_out, T_IN, N_NODE]
        return x_out, attn_output


In [17]:
class transformer_layer(nn.Module):

    '''
    Transformer layer definition designed for extracting spatio-temporal features.
    It utilizes an alignment layer followed by a Transformer Encoder to process the input data.

    Parameters:
    c_in: Number of input channels.
    c_out: Number of output channels.
    T_IN: Number of input time steps.
    BATCHSIZE: Batch size.
    N_NODE: Number of nodes (spatial dimension).
    HEAD: Number of attention heads in the Transformer.
    act: Activation function (default is "relu").
    '''

    def __init__(self, c_in, c_out, T_IN, BATCHSIZE, N_NODE, HEAD, act="relu"):
        super(transformer_layer, self).__init__()
        self.act = act  # Activation function
        self.c_in = c_in  # Number of input channels
        self.c_out = c_out  # Number of output channels
        self.T_IN = T_IN  # Number of input time steps
        self.BATCHSIZE = BATCHSIZE  # Batch size
        self.N_NODE = N_NODE  # Number of nodes (spatial dimension)
        self.HEAD = HEAD  # Number of attention heads in the Transformer

        # Align layer to match input and output channels
        self.align = align(c_in, c_out)

        # Transformer Encoder Layer: Processes input data with self-attention mechanism
        self.transformer = nn.TransformerEncoderLayer(d_model=c_out, nhead=HEAD, dim_feedforward=c_out, batch_first=False)

    def forward(self, x):
        # Align the input to have the same number of channels as the output
        x_in = self.align(x)  # Shape: [batch_size, c_out, T_IN, N_NODE]
        # print(f"Align X-in Shape: {x_in.shape}")

        # Permute dimensions to prepare for the Transformer Encoder (time step dimension first)
        x_permut = x_in.permute(2, 0, 3, 1)  # Shape: [T_IN, batch_size, N_NODE, c_out]

        # Reshape to [T_IN, batch_size * N_NODE, c_out] for Transformer processing
        x_reshape = torch.reshape(x_permut, (self.T_IN, -1, self.c_out))
        # print(f"X-in ReShape: {x_reshape.shape}")

        # Pass the reshaped input through the Transformer Encoder
        x_transformer = self.transformer(x_reshape)
        # print(f"X transformer Shape: {x_transformer.shape}")

        # Reshape the Transformer output back to the original dimensions
        x_transformer = x_transformer.reshape(self.T_IN, -1, self.N_NODE, self.c_out).permute(1, 3, 0, 2)
        # print(f"Reshape X transformer Shape: {x_transformer.shape}")

        return x_transformer  # Shape: [batch_size, c_out, T_IN, N_NODE]


In [19]:
class mamba_layer(nn.Module):

    '''
    Mamba layer definition designed for extracting spatio-temporal features.
    This layer combines channel alignment with the Mamba architecture, which includes state-space models (SSMs) and local convolutions.

    Parameters:
    c_in: Number of input channels.
    c_out: Number of output channels.
    T_IN: Number of input time steps.
    BATCHSIZE: Batch size.
    N_NODE: Number of nodes (spatial dimension).
    d_state: SSM state expansion factor.
    d_conv: Local convolution width.
    act: Activation function (default is "relu").
    '''

    def __init__(self, c_in, c_out, T_IN, BATCHSIZE, N_NODE, d_state, d_conv, act="relu"):
        super(mamba_layer, self).__init__()
        self.act = act  # Activation function
        self.c_in = c_in  # Number of input channels
        self.c_out = c_out  # Number of output channels
        self.T_IN = T_IN  # Number of input time steps
        self.BATCHSIZE = BATCHSIZE  # Batch size
        self.N_NODE = N_NODE  # Number of nodes (spatial dimension)

        # Align layer to match input and output channels
        self.align = align(c_in, c_out)

        # Mamba model with state-space model (SSM) and local convolution
        self.mamba = Mamba(d_model=c_out, d_state=d_state, d_conv=d_conv, expand=2)

    def forward(self, x):
        # Align the input to have the same number of channels as the output
        x_in = self.align(x)  # Shape: [batch_size, c_out, T_IN, N_NODE]
        # print(f"Align X-in Shape: {x_in.shape}")

        # Permute dimensions to prepare for the Mamba model (time step dimension first)
        x_permut = x_in.permute(2, 0, 3, 1)  # Shape: [T_IN, batch_size, N_NODE, c_out]

        # Reshape to [T_IN, batch_size * N_NODE, c_out] for Mamba processing
        x_reshape = torch.reshape(x_permut, (self.T_IN, -1, self.c_out))
        # print(f"X-in ReShape: {x_reshape.shape}")

        # Pass the reshaped input through the Mamba model
        x_mamba = self.mamba(x_reshape)
        # print(f"X mamba Shape: {x_mamba.shape}")

        # Reshape the Mamba output back to the original dimensions
        x_mamba = x_mamba.reshape(self.T_IN, -1, self.N_NODE, self.c_out).permute(1, 3, 0, 2)
        # print(f"Reshape X mamba Shape: {x_mamba.shape}")

        return x_mamba  # Shape: [batch_size, c_out, T_IN, N_NODE]


### 4.2 spatio_conv_layer

In [20]:
class spatio_conv_layer(nn.Module):
    """
    Spatio-temporal convolution layer that applies graph convolution over spatial dimensions using a learned filter.

    Args:
        ks (int): Kernel size (number of spatial filters).
        c (int): Number of channels (typically batch size).
        Lk (torch.Tensor): Pre-processed Laplacian matrix with shape (3, 207, 207).
    """
    def __init__(self, ks, c, Lk):
        super(spatio_conv_layer, self).__init__()
        self.Lk = Lk  # Pre-processed Laplacian matrix for graph convolution
        self.theta = nn.Parameter(torch.FloatTensor(c, c, ks))  # Learnable filter parameters
        self.b = nn.Parameter(torch.FloatTensor(1, c, 1, 1))  # Bias term for convolution
        self.reset_parameters()

    def reset_parameters(self):
        """
        Initialize the learnable parameters (theta and b) using Kaiming uniform distribution.
        """
        init.kaiming_uniform_(self.theta, a=math.sqrt(5))  # Initialize theta with Kaiming uniform
        fan_in, _ = init._calculate_fan_in_and_fan_out(self.theta)  # Calculate fan-in
        bound = 1 / math.sqrt(fan_in)  # Define bounds for bias initialization
        init.uniform_(self.b, -bound, bound)  # Initialize bias term b uniformly within the defined bounds

    def forward(self, x):
        """
        Perform the forward pass of the spatial convolution.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, input_channels, time_steps, nodes).

        Returns:
            torch.Tensor: Output tensor after applying the graph convolution and ReLU activation.
        """
        # Perform the graph convolution operation
        x_c = torch.einsum("knm,bitm->bitkn", self.Lk, x)  # Apply Laplacian matrix to the input tensor
        x_gc = torch.einsum("iok,bitkn->botn", self.theta, x_c) + self.b  # Apply the learned filter and add bias

        # Apply ReLU activation and residual connection
        return torch.relu(x_gc + x)


### 4.3 Output Layer


In [21]:
class output_layer(nn.Module):
    """
    Output layer for spatio-temporal prediction.

    Args:
        c (int): Number of input channels.
        T_IN (int): Number of input time steps.
        n (int): Number of nodes.
    """
    def __init__(self, c, T_IN, n):
        super(output_layer, self).__init__()
        # First temporal convolutional layer with GLU activation
        # Kernel size is equal to the length of the input time steps
        self.tconv1 = temporal_conv_layer(T_IN, c, c, "GLU")

        # Layer normalization, applied across nodes and channels
        self.ln = nn.LayerNorm([n, c])

        # Second temporal convolutional layer with sigmoid activation
        # Kernel size of 1, output interpreted as a probability
        self.tconv2 = temporal_conv_layer(1, c, c, "sigmoid")

        # Fully connected layer to produce the final prediction
        # Input channels = c, output channels = 1
        self.fc = nn.Conv2d(c, 1, 1)

    def forward(self, x):
        """
        Forward pass of the output layer.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, channels, time_steps, nodes).

        Returns:
            torch.Tensor: Output tensor after temporal convolution and fully connected layer.
        """
        # Apply the first temporal convolutional layer with GLU activation
        x_t1 = self.tconv1(x)

        # Apply layer normalization (transpose to match LayerNorm dimensions)
        # Transpose: [batch_size, channels, time_steps, nodes] -> [batch_size, time_steps, nodes, channels]
        # After normalization, transpose back to original shape
        x_ln = self.ln(x_t1.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)

        # Apply the second temporal convolutional layer with sigmoid activation
        x_t2 = self.tconv2(x_ln)

        # Apply the final fully connected layer to produce the output prediction
        return self.fc(x_t2)


In [22]:
class lstm_output_layer(nn.Module):
    """
    LSTM-based output layer for spatio-temporal prediction.

    Args:
        c (int): Number of input channels.
        T_IN (int): Number of input time steps.
        n (int): Number of nodes.
        BATCHSIZE (int): Batch size.
    """
    def __init__(self, c, T_IN, n, BATCHSIZE):
        super(lstm_output_layer, self).__init__()
        # First LSTM-based temporal layer with GLU activation
        # Kernel size is equal to the length of the input time steps
        self.tlstm1 = temporal_lstm_layer(c, c, T_IN, BATCHSIZE, n, act="GLU")

        # Layer normalization, applied across nodes and channels
        self.ln = nn.LayerNorm([n, c])

        # Second LSTM-based temporal layer with sigmoid activation
        # Kernel size of 1, output interpreted as a probability
        self.tlstm2 = temporal_lstm_layer(c, c, T_IN, BATCHSIZE, n, act="sigmoid")

        # Fully connected layer to produce the final prediction
        # Input channels = c, output channels = 1
        self.fc = nn.Conv2d(c, 1, 1)

    def forward(self, x):
        """
        Forward pass of the LSTM-based output layer.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, channels, time_steps, nodes).

        Returns:
            torch.Tensor: Output tensor after LSTM layers and fully connected layer.
        """
        # Apply the first LSTM-based temporal layer with GLU activation
        x_t1 = self.tlstm1(x)

        # Apply layer normalization (transpose to match LayerNorm dimensions)
        # Transpose: [batch_size, channels, time_steps, nodes] -> [batch_size, time_steps, nodes, channels]
        # After normalization, transpose back to original shape
        x_ln = self.ln(x_t1.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)

        # Apply the second LSTM-based temporal layer with sigmoid activation
        x_t2 = self.tlstm2(x_ln)

        # Apply the fully connected layer and return the last time step's output
        # Only the last time step is kept, similar to BERT where only the final token is used for downstream tasks
        return self.fc(x_t2)[:, :, -1:, :]


In [23]:
class attention_lstm_output_layer(nn.Module):
    """
    LSTM-based output layer with attention mechanisms for spatio-temporal prediction.

    Args:
        c (int): Number of input channels.
        T_IN (int): Number of input time steps.
        n (int): Number of nodes.
        BATCHSIZE (int): Batch size.
    """
    def __init__(self, c, T_IN, n, BATCHSIZE):
        super(attention_lstm_output_layer, self).__init__()
        # First LSTM-based attention layer with GLU activation
        # Kernel size is equal to the length of the input time steps
        self.tlstm1 = attention_lstm_layer(c, c, T_IN, BATCHSIZE, n, act="GLU")

        # Layer normalization, applied across nodes and channels
        self.ln = nn.LayerNorm([n, c])

        # Second LSTM-based attention layer with sigmoid activation
        # Kernel size of 1, output interpreted as a probability
        self.tlstm2 = attention_lstm_layer(c, c, T_IN, BATCHSIZE, n, act="sigmoid")

        # Fully connected layer to produce the final prediction
        # Input channels = 2 * c, output channels = 1
        # `2 * c` is used because the attention outputs from the two layers are concatenated
        self.fc = nn.Conv2d(2 * c, 1, 1)

    def forward(self, x):
        """
        Forward pass of the LSTM-based output layer with attention mechanisms.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, channels, time_steps, nodes).

        Returns:
            torch.Tensor: Output tensor after LSTM attention layers and fully connected layer.
        """
        # Apply the first LSTM-based attention layer with GLU activation
        x_t1, atten1 = self.tlstm1(x)

        # Apply layer normalization (transpose to match LayerNorm dimensions)
        # Transpose: [batch_size, channels, time_steps, nodes] -> [batch_size, time_steps, nodes, channels]
        # After normalization, transpose back to original shape
        x_ln = self.ln(x_t1.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)

        # Apply the second LSTM-based attention layer with sigmoid activation
        x_t2, atten2 = self.tlstm2(x_ln)

        # Concatenate the attention outputs along the node dimension
        z = torch.cat([atten1, atten2], dim=2)  # Shape: [batch_size, nodes, 2 * channels]

        # Use the attention-weighted sum as the output, replacing the previous method of using only the last time step
        return self.fc(x_t2)


In [24]:
class gru_output_layer(nn.Module):
    """
    GRU-based output layer for spatio-temporal prediction.

    Args:
        c (int): Number of input channels.
        T_IN (int): Number of input time steps.
        n (int): Number of nodes.
        BATCHSIZE (int): Batch size.
    """
    def __init__(self, c, T_IN, n, BATCHSIZE):
        super(gru_output_layer, self).__init__()
        # First GRU-based layer with GLU activation
        # Kernel size is equal to the length of the input time steps
        self.tgru1 = temporal_gru_layer(c, c, T_IN, BATCHSIZE, n, act="GLU")

        # Layer normalization, applied across nodes and channels
        self.ln = nn.LayerNorm([n, c])

        # Second GRU-based layer with sigmoid activation
        # Kernel size of 1, output interpreted as a probability
        self.tgru2 = temporal_gru_layer(c, c, T_IN, BATCHSIZE, n, act="sigmoid")

        # Fully connected layer to produce the final prediction
        # Input channels = c, output channels = 1
        # The final prediction represents the forecasted traffic flow value
        self.fc = nn.Conv2d(c, 1, 1)

    def forward(self, x):
        """
        Forward pass of the GRU-based output layer.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, channels, time_steps, nodes).

        Returns:
            torch.Tensor: Output tensor after GRU layers and fully connected layer.
        """
        # Apply the first GRU-based layer with GLU activation
        x_t1 = self.tgru1(x)

        # Apply layer normalization
        # Transpose: [batch_size, channels, time_steps, nodes] -> [batch_size, time_steps, nodes, channels]
        # After normalization, transpose back to original shape
        x_ln = self.ln(x_t1.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)

        # Apply the second GRU-based layer with sigmoid activation
        x_t2 = self.tgru2(x_ln)

        # Return the output of the fully connected layer
        # Only the last time step's output is used for the final prediction
        # Similar to how BERT uses only the last token's output for downstream tasks
        return self.fc(x_t2)[:, :, -1:, :]


In [25]:
class transformer_output_layer(nn.Module):
    """
    Transformer-based output layer for spatio-temporal prediction.

    Args:
        c (int): Number of input channels.
        T_IN (int): Number of input time steps.
        n (int): Number of nodes.
        BATCHSIZE (int): Batch size.
        HEAD (int): Number of attention heads in the Transformer.
    """
    def __init__(self, c, T_IN, n, BATCHSIZE, HEAD):
        super(transformer_output_layer, self).__init__()
        # First Transformer-based layer
        # Kernel size is equal to the length of the input time steps, channels are adjusted to c
        self.trans1 = transformer_layer(c, c, T_IN, BATCHSIZE, n, HEAD)

        # Layer normalization, applied across nodes and channels
        self.ln = nn.LayerNorm([n, c])

        # Second Transformer-based layer with similar settings
        self.trans2 = transformer_layer(c, c, T_IN, BATCHSIZE, n, HEAD)

        # Fully connected layer to produce the final prediction
        # Input channels = c, output channels = 1
        # The final prediction represents the forecasted traffic flow value
        self.fc = nn.Conv2d(c, 1, 1)

    def forward(self, x):
        """
        Forward pass of the Transformer-based output layer.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, channels, time_steps, nodes).

        Returns:
            torch.Tensor: Output tensor after Transformer layers and fully connected layer.
        """
        # Apply the first Transformer-based layer
        x_t1 = self.trans1(x)

        # Apply layer normalization
        # Transpose: [batch_size, channels, time_steps, nodes] -> [batch_size, time_steps, nodes, channels]
        # After normalization, transpose back to original shape
        x_ln = self.ln(x_t1.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)

        # Apply the second Transformer-based layer
        x_t2 = self.trans2(x_ln)

        # Return the output of the fully connected layer
        # Only the last time step's output is used for the final prediction
        # Similar to how BERT uses only the last token's output for downstream tasks
        return self.fc(x_t2)[:, :, -1:, :]


In [26]:
class mamba_output_layer(nn.Module):
    """
    Mamba-based output layer for spatio-temporal prediction.

    Args:
        c (int): Number of input channels.
        T_IN (int): Number of input time steps.
        n (int): Number of nodes.
        BATCHSIZE (int): Batch size.
        d_state (int): State dimension for the Mamba layer.
        d_conv (int): Convolutional dimension for the Mamba layer.
    """
    def __init__(self, c, T_IN, n, BATCHSIZE, d_state, d_conv):
        super(mamba_output_layer, self).__init__()
        # First Mamba-based layer
        # Kernel size is equal to the length of the input time steps, channels are adjusted to c
        self.mamba1 = mamba_layer(c, c, T_IN, BATCHSIZE, n, d_state, d_conv)

        # Layer normalization, applied across nodes and channels
        self.ln = nn.LayerNorm([n, c])

        # Second Mamba-based layer with similar settings
        self.mamba2 = mamba_layer(c, c, T_IN, BATCHSIZE, n, d_state, d_conv)

        # Fully connected layer to produce the final prediction
        # Input channels = c, output channels = 1
        # The final prediction represents the forecasted traffic flow value
        self.fc = nn.Conv2d(c, 1, 1)

    def forward(self, x):
        """
        Forward pass of the Mamba-based output layer.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, channels, time_steps, nodes).

        Returns:
            torch.Tensor: Output tensor after Mamba layers and fully connected layer.
        """
        # Apply the first Mamba-based layer
        x_t1 = self.mamba1(x)

        # Apply layer normalization
        # Transpose: [batch_size, channels, time_steps, nodes] -> [batch_size, time_steps, nodes, channels]
        # After normalization, transpose back to original shape
        x_ln = self.ln(x_t1.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)

        # Apply the second Mamba-based layer
        x_t2 = self.mamba2(x_ln)

        # Return the output of the fully connected layer
        # Only the last time step's output is used for the final prediction
        # Similar to how BERT uses only the last token's output for downstream tasks
        return self.fc(x_t2)[:, :, -1:, :]


### 4.4 Main Model

In [27]:
from re import X

class My_Model(nn.Module):
    def __init__(self, bs, T_IN, n, p, ks=None, kt=None, A=None, BATCHSIZE=None, HEAD=None, d_state=None, d_conv=None):
        super(My_Model, self).__init__()

        if is_TCN:
            print("Creating TCN MODEL!")
            # cin=1, cout=16
            self.tconv1 = temporal_conv_layer(kt, bs[0][0], bs[0][1], "GLU")
            # cin=16, cout=64
            self.tconv2 = temporal_conv_layer(kt, bs[0][1], bs[0][2])
            self.ln1 = nn.LayerNorm([n, bs[0][2]])
            self.dropout1 = nn.Dropout(p)

            # cin=64, cout=16
            self.tconv3 = temporal_conv_layer(kt, bs[1][0], bs[1][1], "GLU")
            # cin=16, cout=64
            self.tconv4 = temporal_conv_layer(kt, bs[1][1], bs[1][2])
            self.ln2 = nn.LayerNorm([n, bs[1][2]])
            self.dropout2 = nn.Dropout(p)

            self.output = output_layer(bs[1][2], T_IN-4*(kt-1), n)

        elif is_STGCN:
            print("Creating STGCN MODEL!")
            # cin=1, cout=16
            self.tconv1 = temporal_conv_layer(kt, bs[0][0], bs[0][1], "GLU")
            # cin=16, cout=16
            self.sconv1 = spatio_conv_layer(ks, bs[0][1], A)
            # cin=16, cout=64
            self.tconv2 = temporal_conv_layer(kt, bs[0][1], bs[0][2])
            self.ln1 = nn.LayerNorm([n, bs[0][2]])
            self.dropout1 = nn.Dropout(p)

            # cin=64, cout=16
            self.tconv3 = temporal_conv_layer(kt, bs[1][0], bs[1][1], "GLU")
            # cin=16, cout=16
            self.sconv2 = spatio_conv_layer(ks, bs[1][1], A)
            # cin=16, cout=64
            self.tconv4 = temporal_conv_layer(kt, bs[1][1], bs[1][2])
            self.ln2 = nn.LayerNorm([n, bs[1][2]])
            self.dropout2 = nn.Dropout(p)

            self.output = output_layer(bs[1][2], T_IN-4*(kt-1), n)

        elif is_dilated_STGCN:
            print("Creating Dilated STGCN MODEL!")
            # cin=1, cout=16
            self.tconv1 = dilated_temporal_conv_layer(kt, bs[0][0], bs[0][1], "GLU")
            # cin=16, cout=16
            self.sconv1 = spatio_conv_layer(ks, bs[0][1], A)
            # cin=16, cout=64
            self.tconv2 = dilated_temporal_conv_layer(kt, bs[0][1], bs[0][2])
            self.ln1 = nn.LayerNorm([n, bs[0][2]])
            self.dropout1 = nn.Dropout(p)

            # cin=64, cout=16
            self.tconv3 = dilated_temporal_conv_layer(kt, bs[1][0], bs[1][1], "GLU")
            # cin=16, cout=16
            self.sconv2 = spatio_conv_layer(ks, bs[1][1], A)
            # cin=16, cout=64
            self.tconv4 = dilated_temporal_conv_layer(kt, bs[1][1], bs[1][2])
            self.ln2 = nn.LayerNorm([n, bs[1][2]])
            self.dropout2 = nn.Dropout(p)

            self.output = output_layer(bs[1][2], T_IN, n)

        elif is_dilated_no0_STGCN:
            print("Creating Dilated STGCN MODEL!")
            # cin=1, cout=16
            self.tconv1 = dilated_no0_temporal_conv_layer(kt, bs[0][0], bs[0][1], "GLU")
            # cin=16, cout=16
            self.sconv1 = spatio_conv_layer(ks, bs[0][1], A)
            # cin=16, cout=64
            self.tconv2 = dilated_no0_temporal_conv_layer(kt, bs[0][1], bs[0][2])
            self.ln1 = nn.LayerNorm([n, bs[0][2]])
            self.dropout1 = nn.Dropout(p)

            # cin=64, cout=16
            self.tconv3 = dilated_no0_temporal_conv_layer(kt, bs[1][0], bs[1][1], "GLU")
            # cin=16, cout=16
            self.sconv2 = spatio_conv_layer(ks, bs[1][1], A)
            # cin=16, cout=64
            self.tconv4 = dilated_no0_temporal_conv_layer(kt, bs[1][1], bs[1][2])
            self.ln2 = nn.LayerNorm([n, bs[1][2]])
            self.dropout2 = nn.Dropout(p)

            self.output = output_layer(bs[1][2], T_IN, n)

        elif is_STG_LSTM or is_STG_BiLSTM:
            if is_STG_LSTM: print("Creating STGraph LSTM MODEL!")
            elif is_STG_BiLSTM :
                print("Creating STGraph BiLSTM MODEL!")
                bs = [[2*ele for ele in c] for c in bs]
                bs[0][0] = 1

            # cin=1, cout=16
            self.tlstm1 = temporal_lstm_layer(bs[0][0], bs[0][1], T_IN, BATCHSIZE, n, act="GLU")
            # cin=16, cout=16
            self.sconv1 = spatio_conv_layer(ks, bs[0][1], A)
            # cin=16, cout=64
            self.tlstm2 = temporal_lstm_layer(bs[0][1], bs[0][2], T_IN, BATCHSIZE, n)
            self.ln1 = nn.LayerNorm([n, bs[0][2]])
            self.dropout1 = nn.Dropout(p)

            # cin=64, cout=16
            self.tlstm3 = temporal_lstm_layer(bs[1][0], bs[1][1], T_IN, BATCHSIZE, n, act="GLU")
            # cin=16, cout=16
            self.sconv2 = spatio_conv_layer(ks, bs[1][1], A)
            # cin=16, cout=64
            self.tlstm4 = temporal_lstm_layer(bs[1][1], bs[1][2], T_IN, BATCHSIZE, n)
            self.ln2 = nn.LayerNorm([n, bs[1][2]])
            self.dropout2 = nn.Dropout(p)

            self.output = lstm_output_layer(bs[1][2], T_IN, n, BATCHSIZE)

        elif is_STG_AttentionLSTM or is_STG_BiAttentionLSTM:
            if is_STG_AttentionLSTM: print("Creating STGraph Attention LSTM MODEL!")
            elif is_STG_BiAttentionLSTM :
                print("Creating STGraph Attention BiLSTM MODEL!")
                bs = [[2*ele for ele in c] for c in bs]
                bs[0][0] = 1

            # cin=1, cout=16
            self.tlstm1 = attention_lstm_layer(bs[0][0], bs[0][1], T_IN, BATCHSIZE, n, act="GLU")
            # cin=16, cout=16
            self.sconv1 = spatio_conv_layer(ks, bs[0][1], A)
            # cin=16, cout=64
            self.tlstm2 = attention_lstm_layer(bs[0][1], bs[0][2], T_IN, BATCHSIZE, n)
            self.ln1 = nn.LayerNorm([n, bs[0][2]])
            self.dropout1 = nn.Dropout(p)

            # cin=64, cout=16
            self.tlstm3 = attention_lstm_layer(bs[1][0], bs[1][1], T_IN, BATCHSIZE, n, act="GLU")
            # cin=16, cout=16
            self.sconv2 = spatio_conv_layer(ks, bs[1][1], A)
            # cin=16, cout=64
            self.tlstm4 = attention_lstm_layer(bs[1][1], bs[1][2], T_IN, BATCHSIZE, n)
            self.ln2 = nn.LayerNorm([n, bs[1][2]])
            self.dropout2 = nn.Dropout(p)

            # self.output = attention_lstm_output_layer(bs[1][2], T_IN, n, BATCHSIZE)
            self.tlstm5 = attention_lstm_layer(bs[1][2], bs[1][2], T_IN, BATCHSIZE, n, act="GLU")
            self.ln3 = nn.LayerNorm([n, bs[1][2]])

            self.tlstm6 = attention_lstm_layer(bs[1][2], bs[1][2], T_IN, BATCHSIZE, n, act="sigmoid")

            # 全连接层，cin=16， cout=1， 因为我们的最终输出只是一个车流值的预测
            # 这里c*2 是因为我们会concat两个attention后的加群求和，来替换只取最后一列
            bs_total = bs[0][1]+bs[0][2]+bs[1][1]+3*bs[1][2]
            self.fc = nn.Conv2d(bs_total, 1, 1)

        elif is_STG_GRU or is_STG_BiGRU:
            if is_STG_GRU: print("Creating STGraph GRU MODEL!")
            elif is_STG_BiGRU :
                print("Creating STGraph BiGRU MODEL!")
                bs = [[2*ele for ele in c] for c in bs]
                bs[0][0] = 1

            # cin=1, cout=16
            self.tgru1 = temporal_gru_layer(bs[0][0], bs[0][1], T_IN, BATCHSIZE, n, act="GLU")
            # cin=16, cout=16
            self.sconv1 = spatio_conv_layer(ks, bs[0][1], A)
            # cin=16, cout=64
            self.tgru2 = temporal_gru_layer(bs[0][1], bs[0][2], T_IN, BATCHSIZE, n)
            self.ln1 = nn.LayerNorm([n, bs[0][2]])
            self.dropout1 = nn.Dropout(p)

            # cin=64, cout=16
            self.tgru3 = temporal_gru_layer(bs[1][0], bs[1][1], T_IN, BATCHSIZE, n, act="GLU")
            # cin=16, cout=16
            self.sconv2 = spatio_conv_layer(ks, bs[1][1], A)
            # cin=16, cout=64
            self.tgru4 = temporal_gru_layer(bs[1][1], bs[1][2], T_IN, BATCHSIZE, n)
            self.ln2 = nn.LayerNorm([n, bs[1][2]])
            self.dropout2 = nn.Dropout(p)

            self.output = gru_output_layer(bs[1][2], T_IN, n, BATCHSIZE)

        elif is_STG_AttentionGRU or is_STG_BiAttentionGRU:
            if is_STG_AttentionGRU: print("Creating STGraph Attention GRU MODEL!")
            elif is_STG_BiAttentionGRU :
                print("Creating STGraph Attention BiGRU MODEL!")
                bs = [[2*ele for ele in c] for c in bs]
                bs[0][0] = 1

            # cin=1, cout=16
            self.tgru1 = attention_gru_layer(bs[0][0], bs[0][1], T_IN, BATCHSIZE, n, act="GLU")
            # cin=16, cout=16
            self.sconv1 = spatio_conv_layer(ks, bs[0][1], A)
            # cin=16, cout=64
            self.tgru2 = attention_gru_layer(bs[0][1], bs[0][2], T_IN, BATCHSIZE, n)
            self.ln1 = nn.LayerNorm([n, bs[0][2]])
            self.dropout1 = nn.Dropout(p)

            # cin=64, cout=16
            self.tgru3 = attention_gru_layer(bs[1][0], bs[1][1], T_IN, BATCHSIZE, n, act="GLU")
            # cin=16, cout=16
            self.sconv2 = spatio_conv_layer(ks, bs[1][1], A)
            # cin=16, cout=64
            self.tgru4 = attention_gru_layer(bs[1][1], bs[1][2], T_IN, BATCHSIZE, n)
            self.ln2 = nn.LayerNorm([n, bs[1][2]])
            self.dropout2 = nn.Dropout(p)

            # self.output = attention_lstm_output_layer(bs[1][2], T_IN, n, BATCHSIZE)
            self.tgru5 = attention_gru_layer(bs[1][2], bs[1][2], T_IN, BATCHSIZE, n, act="GLU")
            self.ln3 = nn.LayerNorm([n, bs[1][2]])

            self.tgru6 = attention_gru_layer(bs[1][2], bs[1][2], T_IN, BATCHSIZE, n, act="sigmoid")

            # Fully-connected layer, cin=16, cout=1, because our final output is just one prediction of the traffic values.
            # c*2 here because we'll concatenate the sum of the two attentions and replace it with just the last column.
            bs_total = bs[0][1]+bs[0][2]+bs[1][1]+3*bs[1][2]
            self.fc = nn.Conv2d(bs_total, 1, 1)

        elif is_STG_TRANSFORMER:
            print("Creating STGraph TRANSFORMER MODEL!")
            # cin=1, cout=16
            self.trans1 = transformer_layer(bs[0][0], bs[0][1], T_IN, BATCHSIZE, n, HEAD)
            # cin=16, cout=16
            self.sconv1 = spatio_conv_layer(ks, bs[0][1], A)
            # cin=16, cout=64
            self.trans2 = transformer_layer(bs[0][1], bs[0][2], T_IN, BATCHSIZE, n, HEAD)
            self.ln1 = nn.LayerNorm([n, bs[0][2]])
            self.dropout1 = nn.Dropout(p)

            # cin=64, cout=16
            self.trans3 = transformer_layer(bs[1][0], bs[1][1], T_IN, BATCHSIZE, n, HEAD)
            # cin=16, cout=16
            self.sconv2 = spatio_conv_layer(ks, bs[1][1], A)
            # cin=16, cout=64
            self.trans4 = transformer_layer(bs[1][1], bs[1][2], T_IN, BATCHSIZE, n, HEAD)
            self.ln2 = nn.LayerNorm([n, bs[1][2]])
            self.dropout2 = nn.Dropout(p)

            self.output = transformer_output_layer(bs[1][2], T_IN, n, BATCHSIZE, HEAD)

        elif is_STG_MAMBA:
            print("Creating STGraph MAMBA MODEL!")
            # cin=1, cout=16
            self.mamba1 = mamba_layer(bs[0][0], bs[0][1], T_IN, BATCHSIZE, n, d_state, d_conv)
            # cin=16, cout=16
            self.sconv1 = spatio_conv_layer(ks, bs[0][1], A)
            # cin=16, cout=64
            self.mamba2 = mamba_layer(bs[0][1], bs[0][2], T_IN, BATCHSIZE, n, d_state, d_conv)
            self.ln1 = nn.LayerNorm([n, bs[0][2]])
            self.dropout1 = nn.Dropout(p)

            # cin=64, cout=16
            self.mamba3 = mamba_layer(bs[1][0], bs[1][1], T_IN, BATCHSIZE, n, d_state, d_conv)
            # cin=16, cout=16
            self.sconv2 = spatio_conv_layer(ks, bs[1][1], A)
            # cin=16, cout=64
            self.mamba4 = mamba_layer(bs[1][1], bs[1][2], T_IN, BATCHSIZE, n, d_state, d_conv)
            self.ln2 = nn.LayerNorm([n, bs[1][2]])
            self.dropout2 = nn.Dropout(p)

            self.output = mamba_output_layer(bs[1][2], T_IN, n, BATCHSIZE, d_state, d_conv)

    def forward(self, x):
        if is_TCN:
            # print("Forward TCN MODEL!")
            x_t1 = self.tconv1(x)
            x_t2 = self.tconv2(x_t1)
            x_ln1 = self.ln1(x_t2.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            x_drop1 = self.dropout1(x_ln1)

            x_t3 = self.tconv3(x_drop1)
            x_t4 = self.tconv4(x_t3)
            x_ln2 = self.ln2(x_t4.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            x_drop2 = self.dropout2(x_ln2)

            return self.output(x_drop2)

        elif is_STGCN:
            # print("Forward STGCN MODEL!")
            x_t1 = self.tconv1(x)
            x_s1 = self.sconv1(x_t1)
            x_t2 = self.tconv2(x_s1)
            x_ln1 = self.ln1(x_t2.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            x_drop1 = self.dropout1(x_ln1)

            x_t3 = self.tconv3(x_drop1)
            x_s2 = self.sconv2(x_t3)
            x_t4 = self.tconv4(x_s2)
            x_ln2 = self.ln2(x_t4.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            x_drop2 = self.dropout2(x_ln2)

            return self.output(x_drop2)

        elif is_dilated_STGCN or is_dilated_no0_STGCN:
            # print("Forward STGCN MODEL!")
            x_t1 = self.tconv1(x)
            # print(f"x_tconv1 Shape: {x_t1.shape}")
            x_s1 = self.sconv1(x_t1)
            # print(f"x_s1 Shape: {x_s1.shape}")
            x_t2 = self.tconv2(x_s1)
            # print(f"x_t2 Shape: {x_t2.shape}")
            x_ln1 = self.ln1(x_t2.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln1 Shape: {x_ln1.shape}")
            x_drop1 = self.dropout1(x_ln1)
            # print(f"x_drop1 Shape: {x_drop1.shape}")

            x_t3 = self.tconv3(x_drop1)
            # print(f"x_t3 Shape: {x_t3.shape}")
            x_s2 = self.sconv2(x_t3)
            # print(f"x_s2 Shape: {x_s2.shape}")
            x_t4 = self.tconv4(x_s2)
            # print(f"x_t4 Shape: {x_t4.shape}")
            x_ln2 = self.ln2(x_t4.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln2 Shape: {x_ln2.shape}")
            x_drop2 = self.dropout2(x_ln2)
            # print(f"x_drop2 Shape: {x_drop2.shape}")

            return self.output(x_drop2)

        elif is_STG_LSTM or is_STG_BiLSTM :
            # print("Forward STGCN MODEL!")
            x = self.tlstm1(x)
            # print(f"x_tconv1 Shape: {x.shape}")
            x = self.sconv1(x)
            # print(f"x_s1 Shape: {x.shape}")
            x = self.tlstm2(x)
            # print(f"x_t2 Shape: {x.shape}")
            x = self.ln1(x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln1 Shape: {x.shape}")
            x = self.dropout1(x)
            # print(f"x_drop1 Shape: {x.shape}")

            x = self.tlstm3(x)
            # print(f"x_t3 Shape: {x.shape}")
            x = self.sconv2(x)
            # print(f"x_s2 Shape: {x.shape}")
            x = self.tlstm4(x)
            # print(f"x_t4 Shape: {x.shape}")
            x = self.ln2(x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln2 Shape: {x.shape}")
            x = self.dropout2(x)
            # print(f"x_drop2 Shape: {x.shape}")

            return self.output(x)

        elif is_STG_AttentionLSTM or is_STG_BiAttentionLSTM:
            # print("Forward STGCN MODEL!")
            x1, atten1 = self.tlstm1(x)
            # print(f"x_tconv1 Shape: {x.shape}")
            x_sconv1 = self.sconv1(x1)
            # print(f"x_s1 Shape: {x.shape}")
            x2, atten2 = self.tlstm2(x_sconv1)
            # print(f"x_t2 Shape: {x.shape}")
            x_ln1 = self.ln1(x2.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln1 Shape: {x.shape}")
            x_drop1 = self.dropout1(x_ln1)
            # print(f"x_drop1 Shape: {x.shape}")

            x3, atten3 = self.tlstm3(x_drop1)
            # print(f"x_t3 Shape: {x.shape}")
            x_sconv2 = self.sconv2(x3)
            # print(f"x_s2 Shape: {x.shape}")
            x4, atten4 = self.tlstm4(x_sconv2)
            # print(f"x_t4 Shape: {x.shape}")
            x_ln2 = self.ln2(x4.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln2 Shape: {x.shape}")
            x_drop2 = self.dropout2(x_ln2)
            # print(f"x_drop2 Shape: {x.shape}")

            x5, atten5 = self.tlstm5(x_drop2)
            x_ln3 = self.ln3(x5.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            x6, atten6 = self.tlstm6(x_ln3)

            z = torch.cat([atten1, atten2, atten3, atten4, atten5, atten6], dim=2) #[64, 207, c_out*6]
            z = z.permute(0,2,1).unsqueeze(2) #[64, c_out*6, 1, 207]
            # Here we use attentional weighted sums instead of just the last column as the output.
            return self.fc(z)

        elif is_STG_GRU or is_STG_BiGRU:
            # print("Forward STGCN MODEL!")
            x = self.tgru1(x)
            # print(f"x_tconv1 Shape: {x.shape}")
            x = self.sconv1(x)
            # print(f"x_s1 Shape: {x.shape}")
            x = self.tgru2(x)
            # print(f"x_t2 Shape: {x.shape}")
            x = self.ln1(x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln1 Shape: {x.shape}")
            x = self.dropout1(x)
            # print(f"x_drop1 Shape: {x.shape}")

            x = self.tgru3(x)
            # print(f"x_t3 Shape: {x.shape}")
            x = self.sconv2(x)
            # print(f"x_s2 Shape: {x.shape}")
            x = self.tgru4(x)
            # print(f"x_t4 Shape: {x.shape}")
            x = self.ln2(x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln2 Shape: {x.shape}")
            x = self.dropout2(x)
            # print(f"x_drop2 Shape: {x.shape}")

            return self.output(x)

        elif is_STG_AttentionGRU or is_STG_BiAttentionGRU:
            # print("Forward STGCN MODEL!")
            x1, atten1 = self.tgru1(x)
            # print(f"x_tconv1 Shape: {x.shape}")
            x_sconv1 = self.sconv1(x1)
            # print(f"x_s1 Shape: {x.shape}")
            x2, atten2 = self.tgru2(x_sconv1)
            # print(f"x_t2 Shape: {x.shape}")
            x_ln1 = self.ln1(x2.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln1 Shape: {x.shape}")
            x_drop1 = self.dropout1(x_ln1)
            # print(f"x_drop1 Shape: {x.shape}")

            x3, atten3 = self.tgru3(x_drop1)
            # print(f"x_t3 Shape: {x.shape}")
            x_sconv2 = self.sconv2(x3)
            # print(f"x_s2 Shape: {x.shape}")
            x4, atten4 = self.tgru4(x_sconv2)
            # print(f"x_t4 Shape: {x.shape}")
            x_ln2 = self.ln2(x4.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln2 Shape: {x.shape}")
            x_drop2 = self.dropout2(x_ln2)
            # print(f"x_drop2 Shape: {x.shape}")

            x5, atten5 = self.tgru5(x_drop2)
            x_ln3 = self.ln3(x5.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            x6, atten6 = self.tgru6(x_ln3)

            z = torch.cat([atten1, atten2, atten3, atten4, atten5, atten6], dim=2) #[64, 207, c_out*6]
            z = z.permute(0,2,1).unsqueeze(2) #[64, c_out*6, 1, 207]
            # Here we use attentional weighted sums instead of just the last column as the output.
            return self.fc(z)

        elif is_STG_TRANSFORMER:
            # print("Forward STGCN MODEL!")
            x = self.trans1(x)
            # print(f"x_tconv1 Shape: {x.shape}")
            x = self.sconv1(x)
            # print(f"x_s1 Shape: {x.shape}")
            x = self.trans2(x)
            # print(f"x_t2 Shape: {x.shape}")
            x = self.ln1(x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln1 Shape: {x.shape}")
            x = self.dropout1(x)
            # print(f"x_drop1 Shape: {x.shape}")

            x = self.trans3(x)
            # print(f"x_t3 Shape: {x.shape}")
            x = self.sconv2(x)
            # print(f"x_s2 Shape: {x.shape}")
            x = self.trans4(x)
            # print(f"x_t4 Shape: {x.shape}")
            x = self.ln2(x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln2 Shape: {x.shape}")
            x = self.dropout2(x)
            # print(f"x_drop2 Shape: {x.shape}")

            return self.output(x)

        elif is_STG_MAMBA:
            # print("Forward STGCN MODEL!")
            x = self.mamba1(x)
            # print(f"x_tconv1 Shape: {x.shape}")
            x = self.sconv1(x)
            # print(f"x_s1 Shape: {x.shape}")
            x = self.mamba2(x)
            # print(f"x_t2 Shape: {x.shape}")
            x = self.ln1(x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln1 Shape: {x.shape}")
            x = self.dropout1(x)
            # print(f"x_drop1 Shape: {x.shape}")

            x = self.mamba3(x)
            # print(f"x_t3 Shape: {x.shape}")
            x = self.sconv2(x)
            # print(f"x_s2 Shape: {x.shape}")
            x = self.mamba4(x)
            # print(f"x_t4 Shape: {x.shape}")
            x = self.ln2(x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            # print(f"x_ln2 Shape: {x.shape}")
            x = self.dropout2(x)
            # print(f"x_drop2 Shape: {x.shape}")

            return self.output(x)

# Main Function

In [28]:
FLOWPATH = './metrla_complet/metr-la.h5'
ADJPATH = './metrla_complet/W_metrla.csv'

TRAINRATIO = 0.8
TRAINVALSPLIT = 0.125
TIMESTEP_PER_HOUR = 12
TIMESTEP_IN = 12
TIMESTEP_OUT = 3
BATCHSIZE = 64

########### Process and Split RAW Data ###########
set_seed(100)

data = pd.read_hdf(FLOWPATH)
scaler = StandardScaler()
data = scaler.fit_transform(data)

train_valXS, train_valYS = getXSYS_single(data, 'TRAIN')
testXS, testYS = getXSYS_single(data, 'TEST')

train_iter, val_iter = create_dataset(train_valXS, train_valYS, device, 'TRAIN_VALID')
test_iter = create_dataset(testXS, testYS, device, 'TEST')

Original Train_Valid Dataset size: 27403.
Train Dataset size: 23977.
Valide Dataset size: 3426.
Test Dataset size: 6853.


In [33]:
from operator import truediv
from itertools import filterfalse
set_seed(100)

wb_log = True
EPOCH = 500
models_iter = ["TCN", "STGCN", "dilated_STGCN", "dilated_no0_STGCN"]
# , "TCN", "STGCN" "dilated_STGCN" "dilated_no0_STGCN" "STG_LSTM" "STG_GRU" STG_TRANSFORMER STG_MAMBA  STG_BiLSTM STG_BiGRU STG_AttentionLSTM STG_BiAttentionLSTM
# STG_AttentionGRU STG_BiAttentionGRU

for mi in models_iter:
    if mi == "TCN":
        is_TCN = True
        is_STGCN = False
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = False
        is_STG_LSTM = False
        is_STG_BiLSTM = False
        is_STG_GRU = False
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = False
        print("="*12, f" MODEL {mi} ", "="*12,'\n')
    elif mi == "STGCN":
        is_TCN = False
        is_STGCN = True
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = False
        is_STG_LSTM = False
        is_STG_BiLSTM = False
        is_STG_GRU = False
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = False
        print("\n", "="*12, f" MODEL {mi} ", "="*12,'\n')
    elif mi == "dilated_STGCN":
        is_TCN = False
        is_STGCN = False
        is_dilated_STGCN = True
        is_dilated_no0_STGCN = False
        is_STG_LSTM = False
        is_STG_BiLSTM = False
        is_STG_GRU = False
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = False
    elif mi == "dilated_no0_STGCN":
        is_TCN = False
        is_STGCN = False
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = True
        is_STG_LSTM = False
        is_STG_BiLSTM = False
        is_STG_GRU = False
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = False
    elif mi == "STG_LSTM":
        is_TCN = False
        is_STGCN = False
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = False
        is_STG_LSTM = True
        is_STG_BiLSTM = False
        is_STG_GRU = False
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = False
    elif mi == "STG_BiLSTM":
        is_TCN = False
        is_STGCN = False
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = False
        is_STG_LSTM = False
        is_STG_BiLSTM = True
        is_STG_GRU = False
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = False
    elif mi == "STG_AttentionLSTM":
        is_TCN = False
        is_STGCN = False
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = False
        is_STG_LSTM = False
        is_STG_BiLSTM = False
        is_STG_GRU = False
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = True
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = False
    elif mi == "STG_BiAttentionLSTM":
        is_TCN = False
        is_STGCN = False
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = False
        is_STG_LSTM = False
        is_STG_BiLSTM = False
        is_STG_GRU = False
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = True
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = False
    elif mi == "STG_GRU":
        is_TCN = False
        is_STGCN = False
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = False
        is_STG_LSTM = False
        is_STG_BiLSTM = False
        is_STG_GRU = True
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = False
    elif mi == "STG_BiGRU":
        is_TCN = False
        is_STGCN = False
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = False
        is_STG_LSTM = False
        is_STG_BiLSTM = False
        is_STG_GRU = False
        is_STG_BiGRU = True
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = False
    elif mi == "STG_AttentionGRU":
        is_TCN = False
        is_STGCN = False
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = False
        is_STG_LSTM = False
        is_STG_BiLSTM = False
        is_STG_GRU = False
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = True
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = False
    elif mi == "STG_BiAttentionGRU":
        is_TCN = False
        is_STGCN = False
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = False
        is_STG_LSTM = False
        is_STG_BiLSTM = False
        is_STG_GRU = False
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = True
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = False
    elif mi == "STG_TRANSFORMER":
        is_TCN = False
        is_STGCN = False
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = False
        is_STG_LSTM = False
        is_STG_BiLSTM = False
        is_STG_GRU = False
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = True
        is_STG_MAMBA = False
    elif mi == "STG_MAMBA":
        is_TCN = False
        is_STGCN = False
        is_dilated_STGCN = False
        is_dilated_no0_STGCN = False
        is_STG_LSTM = False
        is_STG_BiLSTM = False
        is_STG_GRU = False
        is_STG_BiGRU = False
        is_STG_AttentionLSTM = False
        is_STG_BiAttentionLSTM = False
        is_STG_AttentionGRU = False
        is_STG_BiAttentionGRU = False
        is_STG_TRANSFORMER = False
        is_STG_MAMBA = True

    # Set up parameters to be stocked in WB
    args = Namespace()
    args.TIMESTEP_IN = TIMESTEP_IN #12
    args.TIMESTEP_OUT = TIMESTEP_OUT #3
    args.DATANAME = 'METR-LA'

    # Number of nodes of highway
    args.N_NODE = 207
    args.EPOCH = EPOCH
    # Input channel size
    args.CHANNEL = 1
    args.BATCHSIZE = BATCHSIZE #64
    args.LEARN = 0.001
    # args.LEARN = 0.001

    args.PATIENCE = 10
    args.OPTIMIZER = 'Adam' # 'RMSprop'
    args.LOSS = 'MAE' #'MSE'

    # Spatial Convolution kernel
    args.ks = 3
    # Temporal convolution kernel
    args.kt = 3
    # Number of hidden channels
    args.bs = [[args.CHANNEL, 16, 64], [64, 16, 64]]
    # args.bs = [[args.CHANNEL, 2, 4], [4, 2, 4]]
    args.HEAD = 1

    # dropout param
    args.p = 0.

    args.d_state = 2
    args.d_conv = 1


    if is_TCN: args.MODEL_NAME = 'TCN'
    elif is_STGCN: args.MODEL_NAME = 'STGCN'
    elif is_dilated_STGCN: args.MODEL_NAME = 'dilated_STGCN'
    elif is_dilated_no0_STGCN: args.MODEL_NAME = 'dilated_no0_STGCN'
    elif is_STG_LSTM: args.MODEL_NAME = 'STG_LSTM'
    elif is_STG_BiLSTM: args.MODEL_NAME = 'STG_BiLSTM'
    elif is_STG_AttentionLSTM: args.MODEL_NAME = 'STG_AttentionLSTM'
    elif is_STG_BiAttentionLSTM: args.MODEL_NAME = 'STG_BiAttentionLSTM'
    elif is_STG_GRU: args.MODEL_NAME = 'STG_GRU'
    elif is_STG_BiGRU: args.MODEL_NAME = 'STG_BiGRU'
    elif is_STG_AttentionGRU: args.MODEL_NAME = 'STG_AttentionGRU'
    elif is_STG_BiAttentionGRU: args.MODEL_NAME = 'STG_BiAttentionGRU'
    elif is_STG_TRANSFORMER: args.MODEL_NAME = f'STG_TRANSFORMER_HEAD{args.HEAD}'
    elif is_STG_MAMBA: args.MODEL_NAME = f'STG_MAMBA_STATE{args.d_state}_CONV{args.d_conv}'

    KEYWORD = args.DATANAME + '_' + args.MODEL_NAME + '_' + datetime.now().strftime("%y%m%d%H%M")
    args.SAVE_PATH = '/content/drive/MyDrive/Personal_Project/Traffic_Prediction/Saves/' + KEYWORD
    print(args.SAVE_PATH)

    if not os.path.exists(args.SAVE_PATH):
        os.makedirs(args.SAVE_PATH)

    if wb_log:
        wandb.login(key=WANDB_TOKEN)
        run = wandb.init(project="Traffic-Prediction-Survey",
                        name=f"{args.MODEL_NAME}-{args.DATANAME}-IN{args.TIMESTEP_IN}-OUT{args.TIMESTEP_OUT}",
                        config=vars(args),
        )

    ############ Initialize model ###########
    if is_TCN:
        model = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=None).to(device)
    elif is_STGCN:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk).to(device)
    elif is_dilated_STGCN:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk).to(device)
    elif is_dilated_no0_STGCN:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk).to(device)
    elif is_STG_LSTM or is_STG_BiLSTM or is_STG_AttentionLSTM or is_STG_BiAttentionLSTM:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk, BATCHSIZE=args.BATCHSIZE).to(device)
    elif is_STG_GRU or is_STG_BiGRU or is_STG_AttentionGRU or is_STG_BiAttentionGRU:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk, BATCHSIZE=args.BATCHSIZE).to(device)
    elif is_STG_TRANSFORMER:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk, BATCHSIZE=args.BATCHSIZE, HEAD=args.HEAD).to(device)
    elif is_STG_MAMBA:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk, BATCHSIZE=args.BATCHSIZE, d_state=args.d_state , d_conv=args.d_conv).to(device)

    if args.LOSS == 'MSE':
        criterion = nn.MSELoss()
    if args.LOSS == 'MAE':
        criterion = nn.L1Loss()

    if args.OPTIMIZER == 'RMSprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=args.LEARN)
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=args.LEARN)

    # if is_STG_TRANSFORMER:
    #     scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

    ########### Train Phase ###########
    min_val_loss = np.inf  # Early Stopping
    wait = 0  # Waiting steps for Early Stopping

    for epoch in tqdm(range(args.EPOCH)):
        starttime = datetime.now()
        loss_sum, n = 0.0, 0
        model.train()
        for x, y in train_iter:
            # Optimiser gradient to zero
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            # if is_STG_TRANSFORMER:
            #     scheduler.step()

            # Recording loss
            loss_sum += loss.item() * y.shape[0] # 64
            n += y.shape[0]

        train_loss = loss_sum / n
        # Calculate the loss of the validation set
        val_loss = evaluateModel(model, criterion, val_iter)

        if val_loss < min_val_loss:
            wait = 0
            min_val_loss = val_loss
            torch.save(model.state_dict(), args.SAVE_PATH +'/'+ f'{args.MODEL_NAME}_IN{args.TIMESTEP_IN}.pt')
        else:
            wait += 1
            if wait == args.PATIENCE:
                print('Early stopping at epoch: %d' % epoch)
                break

        endtime = datetime.now()
        epoch_time = (endtime - starttime).seconds

        if wb_log:
            wandb.log({"epoch": epoch+1,"epoch_time":epoch_time,
                      "train_loss": train_loss, "val_loss": val_loss})

        print(f"epoch{epoch+1}/{args.EPOCH} ", f"time used:{epoch_time} seconds ", f"train loss:{train_loss} ", f"validation loss:{val_loss} ", f"patience:{wait}")

    ########### Test Phase ###########
    print('Model Testing Started ...', time.ctime())
    print('TIMESTEP_IN, TIMESTEP_OUT', args.TIMESTEP_IN, args.TIMESTEP_OUT)

    if is_TCN:
        model_pred = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=None).to(device)
    elif is_STGCN:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model_pred = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk).to(device)
    elif is_dilated_STGCN:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model_pred = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk).to(device)
    elif is_dilated_no0_STGCN:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model_pred = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk).to(device)
    elif is_STG_LSTM or is_STG_BiLSTM or is_STG_AttentionLSTM or is_STG_BiAttentionLSTM:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model_pred = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk, BATCHSIZE=args.BATCHSIZE).to(device)
    elif is_STG_GRU or is_STG_BiGRU or is_STG_AttentionGRU or is_STG_BiAttentionGRU:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model_pred = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk, BATCHSIZE=args.BATCHSIZE).to(device)
    elif is_STG_TRANSFORMER:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model_pred = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk, BATCHSIZE=args.BATCHSIZE, HEAD=args.HEAD).to(device)
    elif is_STG_MAMBA:
        A = pd.read_csv(ADJPATH).values
        W = weight_matrix(A)
        L = scaled_laplacian(W)
        Lk = cheb_poly(L, args.ks)
        Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
        model_pred = My_Model(bs=args.bs, T_IN=args.TIMESTEP_IN, n=args.N_NODE, p=args.p, ks=args.ks, kt=args.kt, A=Lk, BATCHSIZE=args.BATCHSIZE, d_state=args.d_state , d_conv=args.d_conv).to(device)

    model_pred.load_state_dict(torch.load(args.SAVE_PATH +'/'+ f'{args.MODEL_NAME}_IN{args.TIMESTEP_IN}.pt'))

    if args.LOSS == 'MSE':
        criterion = nn.MSELoss()
    if args.LOSS == 'MAE':
        criterion = nn.L1Loss()

    torch_score = evaluateModel(model_pred, criterion, test_iter)
    YS = testYS.copy()
    YS_pred = predictModel(model_pred, test_iter)
    print('YS.shape, YS_pred.shape,', YS.shape, YS_pred.shape)


    YS, YS_pred = np.squeeze(YS), np.squeeze(YS_pred)
    YS = scaler.inverse_transform(YS)
    YS_pred = scaler.inverse_transform(YS_pred)
    print('YS.shape, YS_pred.shape,', YS.shape, YS_pred.shape)

    MSE, RMSE, MAE, MAPE = evaluate_func(YS, YS_pred)
    np.save(args.SAVE_PATH +'/'+ args.MODEL_NAME + '_prediction.npy', YS_pred)
    np.save(args.SAVE_PATH +'/'+args.MODEL_NAME + '_groundtruth.npy', YS)

    print('*' * 40)
    print(f"{args.SAVE_PATH}, TEST, Torch {args.LOSS}: {torch_score}\n")
    print("MSE, RMSE, MAE, MAPE, %.10f, %.10f, %.10f, %.10f\n" % (MSE, RMSE, MAE, MAPE))
    print('Model Testing Ended ...', time.ctime())

    if wb_log:
        wandb.summary["test_torch_MAE"]= torch_score,
        wandb.summary["test_MSE"]= MSE,
        wandb.summary["test_RMSE"]= RMSE,
        wandb.summary["test_MAE"]= MAE,
        wandb.summary["test_MAPE"]= MAPE,
        wandb.finish()

    del model, model_pred
    gc.collect()
    torch.cuda.empty_cache()


============  MODEL TCN  ============ 

/content/drive/MyDrive/Personal_Project/Traffic_Prediction/Saves/METR-LA_TCN_2408262010


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anoif. Use `wandb login --relogin` to force relogin


Creating TCN MODEL!


  0%|          | 0/500 [00:00<?, ?it/s]

epoch1/500  time used:6 seconds  train loss:0.21972030653425345  validation loss:0.18640235527453894  patience:0
epoch2/500  time used:4 seconds  train loss:0.18668174746884025  validation loss:0.18421701748539535  patience:0
epoch3/500  time used:4 seconds  train loss:0.18445797284960413  validation loss:0.18169707757509457  patience:0
epoch4/500  time used:4 seconds  train loss:0.18304801860170666  validation loss:0.1813188063353735  patience:0
epoch5/500  time used:4 seconds  train loss:0.18210676310830753  validation loss:0.17958946305099316  patience:0
epoch6/500  time used:4 seconds  train loss:0.18116086027975917  validation loss:0.1786725593636207  patience:0
epoch7/500  time used:4 seconds  train loss:0.1808779484373657  validation loss:0.18008564609188263  patience:1
epoch8/500  time used:4 seconds  train loss:0.18016001259847564  validation loss:0.17999515635050045  patience:2
epoch9/500  time used:4 seconds  train loss:0.17939149611869398  validation loss:0.1785413817014877

...

  0%|          | 0/500 [00:00<?, ?it/s]

epoch1/500  time used:85 seconds  train loss:0.21538447319364987  validation loss:0.18229174678324261  patience:0
epoch2/500  time used:84 seconds  train loss:0.18593723150142247  validation loss:0.18448152999848702  patience:1
epoch3/500  time used:84 seconds  train loss:0.18303089186629357  validation loss:0.17960122281601745  patience:0
epoch4/500  time used:84 seconds  train loss:0.18266371699885878  validation loss:0.17812934570923508  patience:0
epoch5/500  time used:84 seconds  train loss:0.18075900262622313  validation loss:0.17825500326099553  patience:1
epoch6/500  time used:84 seconds  train loss:0.17999460841841558  validation loss:0.17872167890592408  patience:2
epoch7/500  time used:83 seconds  train loss:0.17944294686322815  validation loss:0.17814843955091475  patience:3
epoch8/500  time used:83 seconds  train loss:0.17910863906760358  validation loss:0.17660549208217383  patience:0
epoch9/500  time used:84 seconds  train loss:0.17795380540296224  validation loss:0.1763

<ipython-input-33-dfcdfef4ed76>:456: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_pred.load_state_dict(torch.load(args.SAVE_PATH +'/'+ f'{args.MODEL_NAME}_IN{args.TIM

YS.shape, YS_pred.shape, (6853, 1, 1, 207) (6853, 1, 1, 207)
YS.shape, YS_pred.shape, (6853, 207) (6853, 207)
****************************************
/content/drive/MyDrive/Personal_Project/Traffic_Prediction/Saves/METR-LA_dilated_no0_STGCN_2408262057, TEST, Torch MAE: 0.18828952379332273

MSE, RMSE, MAE, MAPE, 64.6524029738, 8.0406717987, 3.3989137813, 8.2246107952

Model Testing Ended ... Mon Aug 26 21:44:32 2024


epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
epoch_time,█▅▅▅▅▅▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
train_loss,█▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁
val_loss,▇█▅▅▅▅▅▄▄▄▄▃▃▄▃▃▃▂▃▂▃▂▁▃▃▂▃▂▂▂▂▂
epoch,32
epoch_time,84
train_loss,0.17125
val_loss,0.17373
